In [ ]:
import sys
sys.path.append('../')

import uniflocpy.uTools.data_workflow as data_workflow
import pandas as pd
from uniflocpy.uTools import plotly_workflow
import uniflocpy.uPVT.BlackOil_model as BlackOil_model
import uniflocpy.uValidation.python_api as python_api
import uniflocpy.uTools.uconst as uc
import uniflocpy.uPVT.PVT_fluids as PVT_fluids
import xlwings as xw
import uniflocpy.uMultiphaseFlow.hydr_cor_Beggs_Brill as BB
import uniflocpy.uWell.uPipe as Pipe

In [ ]:
uniflocvba = python_api.API('E:\\Git\\unifloc_vba\\UniflocVBA_7.xlam')

In [ ]:
uniflocvba.f_bb = uniflocvba.book.macro('unf_BegsBrillGradient')

In [ ]:
def convert_result_to_df(result):
    this_dict = {
    "p_grad_pam" : uc.atm2Pa(result[0]),
    "density_part_pam": uc.atm2Pa(result[1]),
    "friction_part_pam":uc.atm2Pa(result[2]),
    "acceleration_part_pam":uc.atm2Pa(result[3]),
    "vsl_msec":result[4],
    "vsg_msec":result[5],
    "liquid_content_with_pains_corr":result[6],
    "flow_regime_number":result[7]
    }
    this_df = pd.DataFrame(this_dict, index = [0])
    return this_df

In [ ]:
pipe = Pipe.Pipe(fluid=BlackOil_model.Fluid())

In [ ]:
pipe.calc_p_grad_pam(2,20)

In [ ]:
arr_d_m = pipe.fluid_flow.d_m
arr_theta_deg = pipe.angle_to_horizontal_grad
eps_m = pipe.hydr_cor.epsilon_friction_m
Ql_rc_m3day = pipe.fluid_flow.qliq_m3day
Qg_rc_m3day = pipe.fluid_flow.qgas_m3day
Mul_rc_cP = pipe.fluid_flow.mu_liq_cP
Mug_rc_cP = pipe.fluid_flow.fl.mu_gas_cp
sigma_l_Nm = pipe.fluid_flow.sigma_liq_Nm
rho_lrc_kgm3 = pipe.fluid_flow.rho_liq_kgm3
rho_grc_kgm3  = pipe.fluid_flow.fl.rho_gas_kgm3
Payne_et_all_holdup   = 1, 
Payne_et_all_friction   = 1 
c_calibr_grav   = 1
c_calibr_fric   = 1

In [ ]:
eps_m

In [ ]:
result = uniflocvba.f_bb(arr_d_m , 
                arr_theta_deg,  eps_m  , 
                Ql_rc_m3day ,  Qg_rc_m3day  , 
                Mul_rc_cP  ,  Mug_rc_cP  , 
                sigma_l_Nm  ,
                rho_lrc_kgm3  , 
                rho_grc_kgm3  , 
                #Payne_et_all_holdup   
                #Payne_et_all_friction , 
                #c_calibr_grav   , 
                #c_calibr_fric   
               )



In [ ]:

convert_result_to_df(result)

In [ ]:
python_flow_model_db = data_workflow.Data()
python_flow_model_db.clear_data()
vba_result_df = None
t_c =100
pipe.fluid_flow.qliq_on_surface_m3day = 50
pipe.fluid_flow.fl.rsb_m3m3 = 300
for p_bar in range(2, 150, 1):
    pipe.calc_p_grad_pam(p_bar, t_c)
    
    python_flow_model_db.get_data(pipe, object_name='python_flow_model')

    arr_d_m = pipe.fluid_flow.d_m
    arr_theta_deg = pipe.angle_to_horizontal_grad
    eps_m = pipe.hydr_cor.epsilon_friction_m
    Ql_rc_m3day = pipe.fluid_flow.qliq_m3day
    Qg_rc_m3day = pipe.fluid_flow.qgas_m3day
    Mul_rc_cP = pipe.fluid_flow.mu_liq_cP
    Mug_rc_cP = pipe.fluid_flow.fl.mu_gas_cp
    sigma_l_Nm = pipe.fluid_flow.sigma_liq_Nm
    rho_lrc_kgm3 = pipe.fluid_flow.rho_liq_kgm3
    rho_grc_kgm3  = pipe.fluid_flow.fl.rho_gas_kgm3
    Payne_et_all_holdup   = 1, 
    Payne_et_all_friction   = 1 
    c_calibr_grav   = 1
    c_calibr_fric   = 1
    
    vba_result = uniflocvba.f_bb(arr_d_m , 
                arr_theta_deg,  eps_m  , 
                Ql_rc_m3day ,  Qg_rc_m3day  , 
                Mul_rc_cP  ,  Mug_rc_cP  , 
                sigma_l_Nm,
                rho_lrc_kgm3  , 
                rho_grc_kgm3  , 
                #Payne_et_all_holdup   
                #Payne_et_all_friction , 
                #c_calibr_grav   , 
                #c_calibr_fric   
               )
    this_vba_result_df = convert_result_to_df(vba_result)
    try:
        vba_result_df = vba_result_df.append(this_vba_result_df)
    except:
        vba_result_df = this_vba_result_df.copy()

In [ ]:
result_unifloc_python = python_flow_model_db.get_data_as_df()
result_unifloc_python.index = result_unifloc_python['python_flow_model.p_bar']
result_unifloc_python.index.name = 'p_bar'

vba_result_df.index = result_unifloc_python.index
vba_result_df.index.name = 'p_bar'
vba_result_df = vba_result_df.add_prefix('vba_flow_model.')
all_result = result_unifloc_python.join(vba_result_df)

In [ ]:
plotly_workflow.plot_by_patterns(all_result, [['pam','acceleration']], ['density','friction','msec'], swap_xy=False, reversed_y=False,  plot_name='Распределение свойств ГЖС от давления')

In [ ]:
plotly_workflow.plot_by_patterns(all_result, [['pam']], ['density','friction','acceleration','msec'], swap_xy=False, reversed_y=False,  plot_name='Распределение свойств ГЖС от давления')

In [ ]:
plotly_workflow.plot_by_patterns(all_result, [['friction','pam']], ['coooo'], swap_xy=False, reversed_y=False,  plot_name='Распределение свойств ГЖС от давления')

In [ ]:
plotly_workflow.plot_by_patterns(all_result, [['density','pam']], ['coooo'], swap_xy=False, reversed_y=False,  plot_name='Распределение свойств ГЖС от давления')

In [ ]:
plotly_workflow.plot_by_patterns(all_result, ['regime'], ['coooo'], swap_xy=False, reversed_y=False,  plot_name='Распределение свойств ГЖС от давления')

In [ ]:
plotly_workflow.plot_by_patterns(all_result, [['liquid_content_','ains']], ['coooo'], swap_xy=False, reversed_y=False,  plot_name='Распределение свойств ГЖС от давления')

In [ ]:
plotly_workflow.plot_by_patterns(all_result, [['vsl']], ['coooo'], swap_xy=False, reversed_y=False,  plot_name='Распределение свойств ГЖС от давления')

In [ ]:
plotly_workflow.plot_by_patterns(all_result, [['vsg']], ['coooo'], swap_xy=False, reversed_y=False,  plot_name='Распределение свойств ГЖС от давления')